In [48]:
# imports
import torch
from torch import nn
from torchvision.models import resnet18
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import platform
from torch.autograd import Variable
import time
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import  FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import glob
import os
from sklearn import metrics

# vis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import sklearn.metrics
from math import ceil
import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import PIL

In [49]:
# import dataset
train = pd.read_csv('../input/train_bbox.csv')

## Generate Paths

In [51]:
# train directory
TRAIN_DIR = "C:\\Users\\Admin\\Git\\SIIM\\train\\"
# paths
paths = list()
# create path column
for _id in tqdm(train['StudyInstanceUID']):
    try:
        paths.append(glob.glob(os.path.join(TRAIN_DIR, _id +"/*/*"))[0])
    except:
        paths.append('')
train['path'] = paths
train.head()

100%|█████████████████████████████████████████████████████████| 7853/7853 [00:00<00:00, 10133.50it/s]


,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,OpacityCount,x,y,width,height,path
0,000a312787f2_image,"{'x': 789.28836, 'y': 582.43035, 'width': 1026...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,2,789.28836,582.43035,1026.65662,1917.30292,C:\Users\Admin\Git\SIIM\train\5776db0cec75\814...
1,000a312787f2_image,"{'x': 2245.91208, 'y': 591.20528, 'width': 109...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,2,2245.91208,591.20528,1094.66162,1761.54944,C:\Users\Admin\Git\SIIM\train\5776db0cec75\814...
2,0012ff7358bc_image,"{'x': 677.42216, 'y': 197.97662, 'width': 867....",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0,2,677.42216,197.97662,867.79767,999.78214,C:\Users\Admin\Git\SIIM\train\9d514ce429a7\228...
3,0012ff7358bc_image,"{'x': 1792.69064, 'y': 402.5525, 'width': 617....",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0,2,1792.69064,402.55250,617.02734,1204.35800,C:\Users\Admin\Git\SIIM\train\9d514ce429a7\228...
4,001398f4ff4f_image,"{'x': 2729, 'y': 2181.33331, 'width': 948.0001...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1,1,2729.00000,2181.33331,948.00012,604.00000,C:\Users\Admin\Git\SIIM\train\28dddc8559b2\4d4...
